# Kiểm tra dataset Việt Nam TSR 

In [1]:
!ls /kaggle/input/build-dataset-vn-tsd/vntsr_yolo

images	labels


# Import và Cấu hình

In [2]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader, Dataset
import os
import cv2
import numpy as np
from glob import glob
import albumentations as A # Dùng thư viện này để augment cho dễ và mạnh
from albumentations.pytorch.transforms import ToTensorV2
from tqdm import tqdm
from torchvision.models.detection import ssd300_vgg16

# Cấu hình Hyperparameters
BATCH_SIZE = 8 # Faster R-CNN ăn VRAM nhiều hơn YOLO, nên để batch nhỏ
NUM_EPOCHS = 30
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
NUM_CLASSES = 2 # 1 class biển báo + 1 class background (bắt buộc của Faster R-CNN)

# Đường dẫn (Sửa lại nếu cần)
TRAIN_DIR = '/kaggle/input/build-dataset-vn-tsd/vntsr_yolo/images/train'
VAL_DIR = '/kaggle/input/build-dataset-vn-tsd/vntsr_yolo/images/test' # Dùng tập test làm val

# Xây dựng Dataset Class 

In [3]:
class YoloDataset(Dataset):
    def __init__(self, root_dir, transforms=None):
        self.root_dir = root_dir
        self.transforms = transforms
        self.imgs = glob(os.path.join(root_dir, '*g'))

    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        label_path = img_path.replace('images', 'labels').replace('.jpg', '.txt').replace('.png', '.txt')

        boxes = []
        labels = []

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    parts = list(map(float, line.strip().split()))
                    class_id = int(parts[0])
                    x_c, y_c, bbox_w, bbox_h = parts[1:]

                    # Convert YOLO -> Pascal VOC
                    x_min = (x_c - bbox_w / 2) * w
                    y_min = (y_c - bbox_h / 2) * h
                    x_max = (x_c + bbox_w / 2) * w
                    y_max = (y_c + bbox_h / 2) * h
                    
                    # Kẹp giá trị để không bao giờ vượt quá kích thước ảnh
                    x_min = max(0, min(x_min, w - 1))
                    y_min = max(0, min(y_min, h - 1))
                    x_max = max(0, min(x_max, w - 1))
                    y_max = max(0, min(y_max, h - 1))
                    
                    # Kiểm tra box hợp lệ (diện tích > 0) thì mới lấy
                    if x_max > x_min and y_max > y_min:
                        boxes.append([x_min, y_min, x_max, y_max])
                        labels.append(1) 

        target = {}
        
        if self.transforms:
            # Nếu sau khi clip mà không còn box nào (hoặc file txt rỗng)
            if len(boxes) > 0:
                try:
                    transformed = self.transforms(image=img, bboxes=boxes, labels=labels)
                    img = transformed['image']
                    boxes = transformed['bboxes']
                    labels = transformed['labels']
                except ValueError as e:
                    # Backup: Nếu Augmentation vẫn lỗi (do box quá nhỏ), ta bỏ qua box đó
                    print(f"Warning: Lỗi Augmentation tại ảnh {img_path}, bỏ qua box. Error: {e}")
                    boxes = []
                    labels = []
                    # Cần convert img sang tensor thủ công vì transforms thất bại
                    img = A.Compose([A.Resize(640, 640), ToTensorV2()])(image=img)['image']
            else:
                # Nếu không có box, chỉ resize ảnh
                img = A.Compose([A.Resize(640, 640), ToTensorV2()])(image=img)['image']
        
        # Xử lý kết quả đầu ra
        if len(boxes) > 0:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
        else:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)

        img = img.float() / 255.0

        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = torch.tensor([idx])
        
        if len(boxes) > 0:
            target["area"] = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        else:
            target["area"] = torch.as_tensor([], dtype=torch.float32)
            
        target["iscrowd"] = torch.zeros((len(labels),), dtype=torch.int64)

        return img, target

    def __len__(self):
        return len(self.imgs)

# Hàm collate để gom batch (vì số lượng box mỗi ảnh khác nhau)
def collate_fn(batch):
    return tuple(zip(*batch))

# Định nghĩa Augmentation

In [4]:
def get_train_transform():
    return A.Compose([
        # Giả lập Scale
        A.RandomScale(scale_limit=0.25, p=0.25), 
        # Giả lập HSV (hsv_h=0.015, s=0.7, v=0.4 ~ xấp xỉ các giá trị bên dưới)
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
        # Resize về cố định để train theo batch
        A.Resize(height=640, width=640),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

def get_val_transform():
    return A.Compose([
        A.Resize(height=640, width=640),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

# Dtaaloader

In [5]:
# 3. Chuẩn bị Data Loaders
train_dataset = YoloDataset(TRAIN_DIR, transforms=get_train_transform())
val_dataset = YoloDataset(VAL_DIR, transforms=get_val_transform())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=2)

# Load Model và Train Faster R-CNN resnet50

In [6]:
# 1. Load Model Pre-trained
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# 2. Thay đổi đầu ra (Head) cho phù hợp số class của bạn
# (Mặc định COCO là 91, ta đổi thành 2: Background + Traffic Sign)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES)

model.to(DEVICE)

# 4. Optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=1e-4, momentum=0.9, weight_decay=0.0005)
# Learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# 5. Training Loop
print(f"Bắt đầu training Faster R-CNN trên {DEVICE}...")

for epoch in range(NUM_EPOCHS):
    model.train()
    epoch_loss = 0
    
    # Tạo thanh progress bar cho epoch hiện tại
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}", leave=True)
    
    for images, targets in loop:
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        epoch_loss += losses.item()
        
        # Cập nhật loss hiện tại lên thanh progress bar
        loop.set_postfix(loss=losses.item())

    lr_scheduler.step()
    
    # In loss trung bình của cả epoch
    print(f"Epoch {epoch+1} Average Loss: {epoch_loss/len(train_loader):.4f}")

    # Lưu model
    torch.save(model.state_dict(), 'faster_rcnn_custom.pth')
    print("Đã lưu model thành công!")
    

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 212MB/s]


Bắt đầu training Faster R-CNN trên cuda...


Epoch 1/30: 100%|██████████| 319/319 [04:27<00:00,  1.19it/s, loss=0.308]


Epoch 1 Average Loss: 0.3491
Đã lưu model thành công!


Epoch 2/30: 100%|██████████| 319/319 [04:25<00:00,  1.20it/s, loss=0.253]


...

Epoch 28 Average Loss: 0.2020
Đã lưu model thành công!


Epoch 29/30: 100%|██████████| 319/319 [04:25<00:00,  1.20it/s, loss=0.223]


Epoch 29 Average Loss: 0.2022
Đã lưu model thành công!


Epoch 30/30: 100%|██████████| 319/319 [04:25<00:00,  1.20it/s, loss=0.154]


Epoch 30 Average Loss: 0.2016
Đã lưu model thành công!


# Load và train model SSD VGG16

In [7]:
# 2. Load Model SSD300 VGG16
print("Đang khởi tạo model SSD300 VGG16...")
# pretrained_backbone=True: Chỉ load weight của VGG16, phần Head sẽ random init theo num_classes=2
model = ssd300_vgg16(pretrained_backbone=True, num_classes=NUM_CLASSES, trainable_backbone_layers=3)
model.to(DEVICE)

# 3. Setup Optimizer & Scheduler
params = [p for p in model.parameters() if p.requires_grad]
# SSD thường cần learning rate khởi điểm cao hơn Faster R-CNN một chút hoặc tương đương
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)


# 4. Training Loop (SSD)
print(f"Bắt đầu training SSD trên {DEVICE}...")

for epoch in range(NUM_EPOCHS):
    model.train()
    epoch_loss = 0
    
    # Thanh progress bar
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}", leave=True)
    
    for images, targets in loop:
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        # Forward pass
        # SSD trả về dict loss giống Faster R-CNN (bbox_regression, classification)
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # Backward pass
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        epoch_loss += losses.item()
        
        # Update progress bar
        loop.set_postfix(loss=losses.item())

    lr_scheduler.step()
    print(f"Epoch {epoch+1} Average Loss: {epoch_loss/len(train_loader):.4f}")

    # 5. Lưu model
    torch.save(model.state_dict(), 'ssd300_vgg16_custom.pth')
    print("Đã lưu model SSD thành công!")
    

Đang khởi tạo model SSD300 VGG16...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained_backbone' is deprecated since 0.13 and may be removed in the future, please use 'weights_backbone' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights_backbone' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights_backbone=VGG16_Weights.IMAGENET1K_FEATURES`. You can also use `weights_backbone=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16_features-amdegroot-88682ab5.pth" to /root/.cache/torch/hub/checkpoints/vgg16_features-amdegroot-88682ab5.pth
100%|██████████| 528M/528M [00:02<00:00, 205MB/s]


Bắt đầu training SSD trên cuda...


Epoch 1/30: 100%|██████████| 319/319 [01:06<00:00,  4.78it/s, loss=3.59]


Epoch 1 Average Loss: 5.2904
Đã lưu model SSD thành công!


...

Epoch 28 Average Loss: 0.1996
Đã lưu model SSD thành công!


Epoch 29/30: 100%|██████████| 319/319 [01:07<00:00,  4.74it/s, loss=0.166]


Epoch 29 Average Loss: 0.1688
Đã lưu model SSD thành công!


Epoch 30/30: 100%|██████████| 319/319 [01:07<00:00,  4.73it/s, loss=0.131]


Epoch 30 Average Loss: 0.1316
Đã lưu model SSD thành công!


# Inference

In [8]:
import torch, time, os, gc
import numpy as np
import torchvision
from tqdm import tqdm
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FastRCNNPredictor, ssd300_vgg16
from torchvision import transforms

ImportError: cannot import name 'FastRCNNPredictor' from 'torchvision.models.detection' (/usr/local/lib/python3.11/dist-packages/torchvision/models/detection/__init__.py)

## Faster R-CNN

In [ ]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import transforms

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# ==================== HÀM VẼ BBOX ====================
def visualize_boxes(image, boxes, scores, conf_thresh=0.5):
    vis_img = image.copy()
    for box, score in zip(boxes, scores):
        if score >= conf_thresh:
            x1, y1, x2, y2 = box.astype(int)
            color = (0, 255, 0)
            cv2.rectangle(vis_img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(vis_img, f"{score:.2f}", (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2, lineType=cv2.LINE_AA)
    plt.figure(figsize=(10, 8))
    plt.imshow(vis_img)
    plt.axis("off")
    plt.title("📦 Predicted Bounding Boxes")
    plt.show()


# ==================== CẤU HÌNH MODEL ====================
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_CLASSES = 2  # background + traffic_sign

print("🔧 Loading Faster R-CNN ResNet50-FPN...")
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=None)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES)

# Load weights
model.load_state_dict(torch.load("faster_rcnn_custom.pth", map_location=DEVICE))
model.to(DEVICE)
model.eval()
print("✅ Loaded Faster R-CNN weights thành công!")

# ==================== XỬ LÝ ẢNH ====================
test_img = "/kaggle/input/build-dataset-vn-tsd/vntsr_yolo/images/test/0002.jpg"
img = cv2.imread(test_img)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Giữ lại kích thước gốc
orig_h, orig_w = img_rgb.shape[:2]
resize_size = 640

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((resize_size, resize_size))
])
img_tensor = transform(Image.fromarray(img_rgb)).to(DEVICE)

# ==================== INFERENCE ====================
with torch.no_grad():
    preds = model([img_tensor])

pred = preds[0]
boxes = pred["boxes"].cpu().numpy()
scores = pred["scores"].cpu().numpy()

# ==================== SCALE LẠI BBOX THEO ẢNH GỐC ====================
scale_x = orig_w / resize_size
scale_y = orig_h / resize_size
boxes[:, [0, 2]] *= scale_x
boxes[:, [1, 3]] *= scale_y

# ==================== HIỂN THỊ ====================
visualize_boxes(img_rgb, boxes, scores, conf_thresh=0.5)

## SSD

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_CLASSES = 2  # background + traffic_sign

# Khởi tạo model (phải trùng config khi train)
model = ssd300_vgg16(weights=None, num_classes=NUM_CLASSES)
model.load_state_dict(torch.load("/kaggle/input/weightothermodel/pytorch/default/1/ssd300_vgg16_custom.pth", map_location=DEVICE))
model.to(DEVICE)
model.eval()
print("✅ Loaded SSD300 VGG16 weights thành công!")


def visualize_boxes(image, boxes, scores, conf_thresh=0.5):
    vis_img = image.copy()
    for box, score in zip(boxes, scores):
        if score >= conf_thresh:
            x1, y1, x2, y2 = box.astype(int)
            color = (0, 255, 0)
            cv2.rectangle(vis_img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(vis_img, f"{score:.2f}", (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2, lineType=cv2.LINE_AA)
    plt.figure(figsize=(10, 8))
    plt.imshow(vis_img)
    plt.axis("off")
    plt.title("📦 Predicted Bounding Boxes")
    plt.show()

# ==================== XỬ LÝ ẢNH ====================
test_img = "/kaggle/input/build-dataset-vn-tsd/vntsr_yolo/images/test/0002.jpg"
img = cv2.imread(test_img)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Giữ lại kích thước gốc
orig_h, orig_w = img_rgb.shape[:2]
resize_size = 640

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((resize_size, resize_size))
])
img_tensor = transform(Image.fromarray(img_rgb)).to(DEVICE)

# ==================== INFERENCE ====================
with torch.no_grad():
    preds = model([img_tensor])

pred = preds[0]
boxes = pred["boxes"].cpu().numpy()
scores = pred["scores"].cpu().numpy()

# ==================== SCALE LẠI BBOX THEO ẢNH GỐC ====================
scale_x = orig_w / resize_size
scale_y = orig_h / resize_size
boxes[:, [0, 2]] *= scale_x
boxes[:, [1, 3]] *= scale_y

# ==================== HIỂN THỊ ====================
visualize_boxes(img_rgb, boxes, scores, conf_thresh=0.5)